<a href="https://colab.research.google.com/github/Farah-Deeba-UNCC/Introduction-to-ML/blob/main/Notebooks/Exercise_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import torch

In [11]:
# t_c is the temperature in celcius; this is our known output.
t_c = [0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0]
# t_u is the temperature in unknown unit; this is our feature.
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]

# Try yourself: Convert these arrays into tensors.
# t_c =
# t_u =

In [7]:
# model definition based on weight w and bias b
def model(t_u, w, b):
    return w * t_u + b

In [8]:
# Loss definition
def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c)**2
    return squared_diffs.mean()

In [12]:
w = torch.ones(())
b = torch.zeros(())

# t_p is the predicted output for the current value of w and b
t_p = model(t_u, w, b)
# Try yourself: print the predicted values t_p

In [13]:
loss = loss_fn(t_p, t_c)
# Try yourself: print loss

In [19]:
# let's normalize the input; you can use the standard way to do normalization; Here we have done a crude normalization so that our two featues are in the similar range.
t_un = 0.1*t_u

In [35]:
# here we define an optimizer for optimizing the parameters of our linear model
import torch.optim as optim
params = torch.tensor([w,b],requires_grad=True)  # notice that requires_grad defines that it is associated with a graph and gradeint will be computed when we call the step function.
learning_rate = 1e-5
optimizer = optim.SGD([params],lr = learning_rate)
# try yourself: print the params.
# try yourself: print the gradient.

In [ ]:
# Let's do one iteration of training: one forward and one backward pass to update the gradient and the parameters.
t_p = model(t_u,*params)
loss = loss_fn(t_p,t_c)
loss.backward()
# try yourself: print the params. Have the values been updated?
# try yourself: print the gradient. Have the values been updated?
optimizer.step()
# try yourself: print the params. Have the values been updated?
# try yourself: print the gradient. Have the values been updated?



In [40]:
# Now we will do training for 5000 iteration to find the optimum value of the parameters.
def training_loop(n_epochs, optimizer, params, t_u,t_c):
  for epoch in range(1,n_epochs+1):
    t_p = model(t_u, *params)
    loss = loss_fn(t_p,t_c)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 500 == 0:
      print('Epoch %d, Loss %f' % (epoch, float(loss)))
      print(params)
      print(params.grad)
  return params

In [ ]:
params = torch.tensor([w,b],requires_grad=True)
learning_rate = 1e-2
optimizer = optim.SGD([params],lr = learning_rate)
# Try yourself: call training_loop function for 5000 epochs.

# Try yourself: print the params values

## Training and Validation in PyTorch

In [29]:
# splitting a dataset into 8:2 ratio
n_samples = t_u.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]
# Try yourself: print train_indices and val_indices. see if the number of training and validation match with our intended ratio

In [30]:
train_t_u = t_u[train_indices]
train_t_c = t_c[train_indices]

val_t_u = t_u[val_indices]
val_t_c = t_c[val_indices]

In [31]:
# crude normalization
train_t_un = 0.1 * train_t_u
val_t_un = 0.1 * val_t_u

In [41]:
def training_loop(n_epochs, optimizer, params, train_t_u,val_t_u, train_t_c,val_t_c):
  for epoch in range(1,n_epochs+1):

    # Try yourself: identif (i) where we are doing forward pass on the training data,
    # (ii)  where we are doing forward pass on the validation data,
    # (iii) where we are computing the gradient on training data
    # (iv) and where we are updating the paramter values on training data (iii and iV constitute the backward pass)
    # (v) where we are doing the backwass pass on validation data

    train_t_p = model(train_t_u, *params)
    train_loss = loss_fn(train_t_p, train_t_c)

    val_t_p = model(val_t_u, *params)
    val_loss = loss_fn(val_t_p, val_t_c)


    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()

    if epoch<= 3 or epoch % 500 == 0:
      print('Epoch %d, Training Loss %f, Validation Loss %f' % (epoch, float(train_loss), float(val_loss)))
      print(params)
      print(params.grad)
  return params

In [ ]:
params = torch.tensor([1.0,0.0],requires_grad = True)
learning_rate = 1e-2
optimizer = optim.SGD([params],lr = learning_rate)

training_loop(
    n_epochs = 5000,
    optimizer = optimizer,
    params = params,
    train_t_u = train_t_un,
    val_t_u = val_t_un,
    train_t_c = train_t_c,
    val_t_c = val_t_c)

# try yourself: print the final values of the parameters. What will be the predicted output if the temperature at unknow unit is 80.

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
t_p = model(t_un, *params)
fig = plt.figure(dpi=600)
plt.xlabel("Temperature (°Fahrenheit)")
plt.ylabel("Temperature (°Celsius)")
plt.plot(t_u.numpy(), t_p.detach().numpy())
plt.plot(t_u.numpy(), t_c.numpy(), 'o')